# Pytorch
- 구글의 tensorflow와 유사한 딥러닝 라이브러리
- 페이스북 인공지능 연구팀에 의해 주로 개발
- torch
    - 텐서 변환 및 다양한 수학 함수와 클래스가 포함되어 있다.
    - 텐서(Tensor)란?
        - 스칼라, 벡터, 행렬, 다차원 배열등의 다양한 모양의 숫자들을 모아놓은 구조
        - 다차원의 배열을 통칭

- torch.nn
    - 신경망을 구축하기 위한 레이어(층), 활성화 함수, 손실함수 등에 대한 함수와 클래스가 포함되어 있다.
- torch.utils.data
    - 미니 배치 학습을 위한 데이터셋 구성관련 함수와 클래스가 포함되어 있다.
- torch.optim
    - optimizer 관련 함수와 클래스가 포함되어 있다.
- https://pytorch.org/

# 타이타닉 데이터셋을 pytorch로 학습 시켜보기
1. 데이터 전처리
2. 학습 데이터를 텐서로 변경
3. 인공 신경망 모델 객체 생성
4. 하이퍼파라미터(손실함수 및 옵티마이저 등 선택)
5. 학습 및 테스트 예측 loop 구현

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/data/"

SEED = 42 # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft = scaler.transform(train_ft)
test_ft = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"].to_numpy()

train_ft.shape, test_ft.shape, target.shape

Mounted at /content/drive


((916, 10), (393, 10), (916,))

In [2]:
train_ft

array([[0.88726043, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.42377552, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.36114243, 0.375     , 0.11111111, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.44882876, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.473882  , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.23587624, 0.125     , 0.11111111, ..., 0.        , 0.        ,
        1.        ]])

In [3]:
target

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

# 손실함수(loss function)

## 회귀문제에서의 손실함수

In [4]:
import torch

- 회귀 문제에서 정답데이터는 2차원 형식이어야 하고, 데이터 타입은 float32 이어야 한다.

In [5]:
torch.manual_seed(SEED)
x = torch.randn(2,3) # 2개의 샘플, 3개의 피처
y = torch.randn(2,1) # 2개의 정답 데이터, 정답데이터는 2차원 형식이어야 함

In [6]:
torch.manual_seed(SEED)
model = torch.nn.Linear(x.shape[1], 1) # 선형회귀

In [7]:
pred = model(x)
pred

tensor([[0.7090],
        [0.1192]], grad_fn=<AddmmBackward0>)

- MSE

In [8]:
loss_fn = torch.nn.MSELoss() # 손실을 계산할 수 있는 손실함수 객체가 반환됨
loss = loss_fn(pred, y) # 손실함수에는 첫 번째 인수로 예측값, 두 번째 인수로 정답값 전달
loss

tensor(1.4105, grad_fn=<MseLossBackward0>)

In [9]:
loss.backward() # 역전파

- MAE

In [10]:
loss_fn = torch.nn.L1Loss()
loss = loss_fn(pred, y)
loss

tensor(1.1282, grad_fn=<MeanBackward0>)

In [11]:
# loss.backward()

## 분류문제에서의 손실함수

- BCE
    - 이진 분류 문제에서 사용되는 손실함수

In [12]:
x

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])

- 이진 분류 문제에서 정답데이터는 2차원 형식이어야 하고, 데이터 타입은 float32 이어야 한다.

In [13]:
y = torch.Tensor([0,1]).view(-1,1)
y

tensor([[0.],
        [1.]])

In [14]:
model = torch.nn.Linear(x.shape[1], 1)
pred = model(x)
pred

tensor([[0.2456],
        [0.2315]], grad_fn=<AddmmBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과한 경우

In [15]:
sig = torch.nn.Sigmoid() # 시그모이드 함수를 반환하는 클래스
loss_fn = torch.nn.BCELoss()
loss_fn(sig(pred), y)

tensor(0.7038, grad_fn=<BinaryCrossEntropyBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과하지 않은 경우

In [16]:
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn(pred, y)

tensor(0.7038, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

- CE(Cross Entropy)
    - 다중 분류 문제에서 사용되는 손실함수

In [17]:
x = torch.randn(4,2) # 4개의 샘플 2개의 피처
x

tensor([[-1.1229, -0.1863],
        [ 2.2082, -0.6380],
        [ 0.4617,  0.2674],
        [ 0.5349,  0.8094]])

- 다중 분류 문제에서의 정답데이터는 1차원 형태여야 하고, 데이터타입은 int64 형태여야 함

In [18]:
y = torch.tensor([0,1,2,1], dtype=torch.int64)
y.shape, y.dtype

(torch.Size([4]), torch.int64)

In [19]:
model = torch.nn.Linear(x.shape[1], 3)
pred = model(x)
pred

tensor([[ 1.3261, -0.2972,  0.6011],
        [-0.3860, -0.7703,  1.1389],
        [ 0.2938, -0.8065,  0.3900],
        [ 0.0762, -1.0516,  0.0164]], grad_fn=<AddmmBackward0>)

In [20]:
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(pred, y)

tensor(1.3699, grad_fn=<NllLossBackward0>)

# 옵티마이저(Optimizer)
- pytorch 의 옵티마이저의 주요 파라미터
    - 첫 번째 인수로 모델의 파라미터 전달
    - lr: 학습률
    - weight_decay: 가중치 감소(L2 정규화)를 적용하는데 사용되는 계수
        - 기본값은 0: 가중치 감소 x

In [21]:
model.parameters()

<generator object Module.parameters at 0x792820387d80>

In [22]:
optim = torch.optim.SGD(model.parameters(), lr=0.001)

In [23]:
optim.step() # 가중치 업데이트

## momentum

In [24]:
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Adagrad

In [25]:
optim = torch.optim.Adagrad(model.parameters())

## RMSprop

In [26]:
optim = torch.optim.RMSprop(model.parameters())

## Adam

In [27]:
optim = torch.optim.Adam(model.parameters())

In [28]:
# 역전파 후
optim.step()